In [ ]:
import urllib3
import json
import certifi
import pandas as pd
import time
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

In [ ]:
#intial load
http = urllib3.PoolManager(cert_reqs='CERT_REQUIRED', ca_certs=certifi.where())
requestUrl = "http://www.reddit.com/r/wallstreetbets/.json?sort=top&t;=thisweek&limit;=100"
payload = http.request('GET', requestUrl)
rawDataTemp = json.loads(payload.data.decode('utf-8'))
rawData = rawDataTemp['data']['children']

In [ ]:
#loop due to reddit 100 post limit
for i in range(10):
    #reddit only accepts one request every 2 seconds, adds a delay between each loop
    time.sleep(4)
    last = rawData[-1]['data']['name']
    http = urllib3.PoolManager(cert_reqs='CERT_REQUIRED', ca_certs=certifi.where())
    requestUrl = 'https://www.reddit.com/r/wallstreetbets/.json?sort=top&t;=thisweek&limit;=100&after;=%s' % last
    payload = http.request('GET', requestUrl)
    rawDataTemp = json.loads(payload.data.decode('utf-8'))
    rawData += rawDataTemp['data']['children']

In [ ]:
dataNames = [value for value in rawData[0]['data']]

In [ ]:
tmpList = []
for i in range(0, len(rawData)):
    dictionary = {}
    for names in dataNames:
        try:
            dictionary[str(names)] = rawData[i]['data'][str(names)]
        except:
            dictionary[str(names)] = 'None'
    tmpList.append(dictionary)
df=pd.DataFrame(tmpList)

In [ ]:
df.drop(df.columns.difference(['title']), 1, inplace=True)
df['title'] = df['title'].str.upper()
df['title'] = df['title'].str.replace('[^a-zA-Z]', ' ')
df['title'] = df['title'].str.replace('[^\w\s]+', ' ')

tokens = [word_tokenize(i) for i in df['title']]

In [ ]:
dfTokens = pd.dataframe(tokens)
tmpWordList = []

for col in dfTokens:
    tmpWordList.append(dfTokens[col])

words = pd.DataFrame(tmpWordList)

In [ ]:
# filter out stop words
stopWords = set(stopwords.words('english'))
words = [w for w in words if not w in stop_words]